In [56]:
import string
import re
import unidecode
import pandas as pd

In [57]:
df = pd.read_json("../datasets/development.json", orient='records')
df.sample(5).head(5)["description"]


9986    Sobre a vaga \r\n\r\n\r\n    Descricao\r\n\r\n...
457     LiDER DE LOGiSTICA\n\n\nRequisitos: Experienci...
6167    Mpresa terceirizada no ramo de conservacao e l...
7375    Vendedor externo funcoes: prospeccao e visita ...
471     Eletricista - CONTAGEM - MG - 1 VAGA URGENTE  ...
Name: description, dtype: object

### Definindo pre processing especifico para o parser

#### Tira espaços e mantem ":"

In [58]:
def preprocess_parser(rawtxt):
    txt = remove_punctuation_except_colon(rawtxt)
    txt = re.sub(r'^[a-z\u00E0-\u00FCA-Z\u00E0-\u00FC]+$/i', ' ', txt)
    txt = unidecode.unidecode(txt).lower()
    txt = txt.replace(" ", "")
    txt = txt.replace("-","")
    return txt

def remove_punctuation_except_colon(s):
    delimiters = string.punctuation
    delimiters = delimiters.replace(":","")
    delimiters+= '\n\r\t' 												# additions
    this_translation=str.maketrans(delimiters,' '*len(delimiters))
    s= s.translate(this_translation) # removes punctuation
    return s

In [59]:
toy_example = "Isto é só um teste, ok? Local: São Paulo / SP"
preprocess_parser(toy_example)

'istoesoumtesteoklocal:saopaulosp'

In [60]:
cleaner = lambda x : preprocess_parser(x)
df['clean_description'] = df['description'].transform(cleaner)

In [61]:
df.sample(5)["clean_description"]

9032    atividadesprofissionais:fornecersuporteagestao...
2213    consultorcomercialbaurudatadecadastro:escolalo...
4858    atuaracomatividadespertinentesaocargoescolarid...
2751    irarealizarvendasdeemprestimosconsignadoscarto...
4638    :responsabilidades:iraatuarnaconfeccaodeinfrae...
Name: clean_description, dtype: object

### Rodando o parser para gerar os dicionarios

In [66]:
dicionario_locais = {}
dicionario_formacao = {}
lookup_local = ["local:", "localdetrabalho:", "localdavaga:", "cidade:","localizadaem","localdotrabalho:","localizacao:"]
lookup_formacao = ["formacaorequerida:","formacaodesejada:","formacao:","escolaridade:",
                   "escolaridadedesejada:","escolaridaderequerida:"]

for idx, vaga in df.iterrows():
    description = str(vaga["clean_description"])
    for l in lookup_local:
        if(l in description):
            dicionario_locais[idx] = description[(description.index(l) + len(l)):(description.index(l) + len(l) + 7)]

    for lc in lookup_formacao:
        if(lc in description):
            dicionario_formacao[idx] = description[(description.index(lc) + len(lc)):(description.index(lc) + len(lc) + 7)]


### Exemplificando os resultados:

In [67]:
dict(list(dicionario_locais.items())[0:9])

{16: 'analiaf',
 1391: 'riodeja',
 6146: 'riodeja',
 6957: 'ribeira',
 8199: 'riodeja',
 8200: 'riodeja',
 8211: 'atender',
 9629: 'saojose',
 9689: 'saopaul'}

In [68]:
dict(list(dicionario_formacao.items())[0:9])

{16: 'ensinom',
 2056: 'ensinom',
 6153: 'ensinos',
 6162: 'ensinof',
 6169: 'tecnico',
 8199: 'fundame',
 8204: 'ensinos',
 8536: 'ensinom',
 9288: 'ensinom'}

### Checando para quantas vagas as informaçoes foram encontradas

In [69]:
len(dicionario_locais.keys())

2324

In [70]:
len(dicionario_formacao.keys())

1012